In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import pandas as pd
import os
import clip
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Загрузка данных из CSV
df = pd.read_csv('flickr30k_images/results.csv', sep='|')

In [ ]:
df[' comment'] = df[' comment'].astype(str)

In [ ]:
# Максимальная длина текста
max_length = 75

# Функция для укорачивания текста до максимальной длины
def truncate_text(text):
    if len(text) > max_length:
        return text[:max_length]
    else:
        return text

# Примените функцию к вашей колонке с текстами (например, 'text_column')
df[' comment'] = df[' comment'].apply(truncate_text)

In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

#image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)
text = clip.tokenize(df[' comment']).to(device)
batch_size = 8  # Adjust as needed
text_features = []

with torch.no_grad():
    for i in range(0, len(text), batch_size):
        batch_text = text[i:i+batch_size]
        batch_features = model.encode_text(batch_text)
        text_features.append(batch_features)

text_features = torch.cat(text_features)

In [ ]:
import os
import torch
import clip
from PIL import Image
import numpy as np
import os
def encode_images_in_folder(image_folder, model, device):
    # Create a list to store image features
    image_features_list = []
    # Iterate through the images in the folder
    for filename in os.listdir(image_folder):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            # Construct the full path to the image file
            image_path = os.path.join(image_folder, filename)
            # Preprocess and encode the image
            image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
            with torch.no_grad():
                image_features = model.encode_image(image)
            image_features_list.append(image_features)
    # Concatenate the image features
    if image_features_list:
        image_features_tensor = torch.cat(image_features_list, dim=0)
        return image_features_tensor
    else:
        print('No valid images found in the folder.')
        return None
# Usage example
image_folder = 'flickr30k_images/flickr30k_images/flickr30k_images'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, preprocess = clip.load('ViT-B/32', device=device)
image_features = encode_images_in_folder(image_folder, model, device)
if image_features is not None:
    print(f'Encoded features for {len(image_features)} images.')

In [ ]:
image = preprocess(Image.open("flickr30k_images/flickr30k_images/8218549120.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a woman"]).to(device)


with torch.no_grad():
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]

In [ ]:
# Сохранение весов text_features в файл text_features.pt
torch.save(text_features, 'text_features.pt')

# Сохранение весов image_features в файл image_features.pt
torch.save(image_features, 'image_features.pt')